In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

from contextlib import contextmanager
import numpy as np
from copy import deepcopy


# from proposed.utils import get_dist
from LVD.utils import get_dist


import math
from LVD.modules.base import *


No module named 'mjrl'
No module named 'flow'
/home/magenta1223/anaconda3/envs/skill2/lib/python3.8/site-packages/glfw/__init__.py:916: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)
No module named 'dotmap'
pybullet build time: May 20 2022 19:44:17


In [2]:
from typing import Dict


class BaseModule(nn.Module):
    """
    """
    def __init__(self, config):
        super(BaseModule, self).__init__()
        self.set_attrs(config)
        self._device = nn.Parameter(torch.zeros(1))

    # set configs
    def set_attrs(self, config = None):
        if config is not None:
            try:
                for k, v in config.attrs.items():
                    setattr(self, k, deepcopy(v))
            except:
                for k, v in config.items():
                    setattr(self, k, deepcopy(v))           

    def forward(self, x):
        return NotImplementedError

    @property
    def device(self):
        return self._device.device



class SequentialBuilder(BaseModule):
    def __init__(self, config : Dict[str, None]):
        super().__init__(config)
        self.build()
        self.explore = None

    def get(self, name):
        if not name : #name is None or not name:
            return None
        return getattr(self, name)

    def layerbuild(self, attr_list, repeat = None):
        build =  [[ self.get(attr_nm) for attr_nm in attr_list  ]]
        if repeat is not None:
            build = build * repeat
        return build 

    def get_build(self):
        if self.module_type == "rnn":
            build = self.layerbuild(["linear_cls", "in_feature", "hidden_dim", None, "act_cls", "bias"])
            build += self.layerbuild(["rnn_cls", "hidden_dim", "hidden_dim", "n_blocks", "bias", "batch_first", "dropout"])
            build += self.layerbuild(["linear_cls", "hidden_dim", "out_dim", None, None, "false"])

        elif self.module_type == "linear":
            build = self.layerbuild(["linear_cls", "in_feature", "hidden_dim", None, "act_cls", "bias", "dropout"])
            build += self.layerbuild(["linear_cls", "hidden_dim", "hidden_dim", "norm_cls", "act_cls"], self.get("n_blocks"))
            build += self.layerbuild(["linear_cls", "hidden_dim", "out_dim", None, None,  "bias", "dropout"])
        else:
            build = NotImplementedError

        return build

    def build(self):
        build = self.get_build()
        layers = []
        for args in build:
            cls, args = args[0], args[1:]
            layers.append(cls(*args))
        self.layers = nn.ModuleList(layers)


    # -------------------- mdoelign -------------------- # 
    
    def forward(self, x, *args, **kwargs):
        out = x 
        for layer in self.layers:
            out = layer(out)
            if isinstance(out, tuple): # rnn
                out = out[0]
        return out


    def dist(self, *args, detached = False):
        result = self(*args)

        if detached:
            return get_dist(result, tanh = self.tanh), get_dist(result, detached= True, tanh = self.tanh)
        else:
            return get_dist(result, tanh = self.tanh)


In [3]:
prior_config = dict(
    n_blocks = 5, #self.n_processing_layers,
    in_feature =  32, # state_dim + latent_dim 
    hidden_dim = 128, 
    out_dim = 10 * 2,
    norm_cls = nn.LayerNorm,
    act_cls = nn.Mish,
    linear_cls = LinearBlock,
    tanh = True,
    bias = True,
    dropout = 0,
    module_type = "linear"
)


encoder_config = dict(
    n_blocks = 5, #self.n_processing_layers,
    in_feature =  32, # state_dim + latent_dim 
    hidden_dim = 128, 
    out_dim = 10 * 2,
    norm_cls = nn.LayerNorm,
    act_cls = nn.Mish,
    linear_cls = LinearBlock,
    tanh = True,
    bias = True,
    dropout = 0,
    module_type = "rnn",
    batch_first = True,
    rnn_cls = nn.LSTM,
    false = False
)


SequentialBuilder(encoder_config)

SequentialBuilder(
  (layers): ModuleList(
    (0): LinearBlock(
      (layers): Sequential(
        (0): Linear(in_features=32, out_features=128, bias=True)
        (1): Mish(inplace=True)
      )
    )
    (1): LSTM(128, 128, num_layers=5, batch_first=True)
    (2): LinearBlock(
      (layers): Sequential(
        (0): Linear(in_features=128, out_features=20, bias=False)
      )
    )
  )
)

In [4]:
from omegaconf import DictConfig, OmegaConf
import hydra

In [5]:
dir(hydra)

['MissingConfigException',
 'TaskFunction',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_internal',
 'compose',
 'conf',
 'core',
 'errors',
 'grammar',
 'initialize',
 'initialize_config_dir',
 'initialize_config_module',
 'main',
 'plugins',
 'types',
 'utils',
 'version']